# Dilated Neighborhood Attention Transformer (DINAT)

## 模型概述

Dilated Neighborhood Attention Transformer (DiNAT) 是一种创新的分层视觉Transformer，旨在提升深度学习模型的性能，尤其是在视觉识别任务中的表现。与传统Transformer使用的自注意力机制不同，DiNAT引入了Dilated Neighborhood Attention (DiNA)，在无需增加计算量的情况下，将局部注意力机制扩展为稀疏的全局注意力。这一扩展使DiNA能够捕捉更多的全局上下文，指数级地扩大感受野，并有效地建模长距离依赖关系。

<div class="wy-nav-content-img">
    <img src="assets/DiNAT_model_arch.png" width=960px alt="DiNAT的模型架构图">
    <p>图1：DiNAT 架构示意图。</p>
</div>

如上图所示：DiNAT 它首先将输入下采样到其原始空间分辨率的四分之一，然后将它们送入 4 级 DiNA Transformer 编码器。特征图在各级之间被下采样到其空间尺寸的一半，而通道数则翻倍。DiNAT 层与大多数 Transformer 类似：先是注意力机制，接着是带有归一化的多层感知机（MLP），且中间有跳跃连接。它还会在每隔一层时（如右图所示）在局部 NA（非自注意力机制）和稀疏全局 DiNA（动态非自注意力机制）之间进行切换。

DiNAT在其架构中结合了NA和DiNA，从而创建了一个能够保持局部性、保持平移等变性，并在下游视觉任务中实现显著性能提升的Transformer模型。实验表明，与诸如 NAT、Swin 和 ConvNeXt 等强基线模型相比，DiNAT 在各种视觉识别任务中表现出明显的优势。

## DiNAT的核心：扩张的邻域注意力

<div class="wy-nav-content-img">
    <img src="assets/DiNAT_dilatedNA.png" width=900px alt="DiNAT的模型架构图">
    <p>图2：邻域注意力（NA）和扩张邻域注意力（DiNA）中单个像素注意力范围的示意图。</p>
</div>

DiNAT 基于Neighborhood Attention (NA)架构，这是一种专门为计算机视觉任务设计的注意力机制，旨在高效地捕捉图像中像素之间的关系。简单来说，可以把它比作图像中每个像素需要理解并关注其周围像素，以更全面地理解整个图像。以下是NA的主要特性：

* 局部关系：NA捕捉局部关系，使每个像素能够从其周围的邻域中获取信息。这类似于我们首先观察最近的物体来理解场景，然后再考虑整个视野的方式。
* 感受野：NA允许像素扩展其对周围环境的理解，而无需增加过多计算量。它能够动态扩展像素的范围或“注意力范围”，在必要时将更远的邻居纳入其中。

总的来说，NA 将注意力定位在像素的最近邻域上。DiNA 将 NA 的局部注意力扩展为一种约束更少的稀疏全局注意力，且不会增加额外的计算负担。由 NA 和 DiNA 组成的变换器能够保留局部性、保持平移等变性、以指数方式扩展感受野，并捕捉更长距离的相互依赖关系，从而在下游视觉任务中显著提升性能。

## 在 Transformers 中使用 DiNAT

In [1]:
from transformers import AutoImageProcessor, DinatForImageClassification
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = AutoImageProcessor.from_pretrained("shi-labs/dinat-mini-in1k-224")
model = DinatForImageClassification.from_pretrained("shi-labs/dinat-mini-in1k-224")

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# 模型预测1000个ImageNet类别中的一个
predicted_class_idx = logits.argmax(-1).item()
print("预测类别:", model.config.id2label[predicted_class_idx])

| 2024-11-12 12:29:05,310 | [[ natten.functional ]] [ WARNING ]: You're calling NATTEN op `natten.functional.natten2dqkrpb`, which is deprecated in favor of `natten.functional.na2d_qk`. Please consider switching, as this op will be removed soon.
| 2024-11-12 12:29:05,870 | [[ natten.functional ]] [ WARNING ]: You're calling NATTEN op `natten.functional.natten2dav`, which is deprecated in favor of `natten.functional.na2d_av`. Please consider switching, as this op will be removed soon.
| 2024-11-12 12:29:05,998 | [[ natten.functional ]] [ WARNING ]: You're calling NATTEN op `natten.functional.natten2dqkrpb`, which is deprecated in favor of `natten.functional.na2d_qk`. Please consider switching, as this op will be removed soon.
| 2024-11-12 12:29:06,558 | [[ natten.functional ]] [ WARNING ]: You're calling NATTEN op `natten.functional.natten2dav`, which is deprecated in favor of `natten.functional.na2d_av`. Please consider switching, as this op will be removed soon.
| 2024-11-12 12:29:06,

预测类别: tabby, tabby cat
